# **Tweet Classifier**

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('TruthSeeker2023/Truth_Seeker_Model_Dataset.csv')
df.head()

,Unnamed: 0,author,statement,target,BinaryNumTarget,manual_keywords,tweet,5_label_majority_answer,3_label_majority_answer
0,0,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Mostly Agree,Agree
1,1,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,NO MAJORITY,Agree
2,2,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",THE SUPREME COURT is siding with super rich pr...,Agree,Agree
3,3,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders\n\nBroken campaign promi...,Mostly Agree,Agree
4,4,D.L. Davis,End of eviction moratorium means millions of A...,True,1.0,"Americans, eviction moratorium",@OhComfy I agree. The confluence of events rig...,Agree,Agree


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134198 entries, 0 to 134197
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               134198 non-null  int64  
 1   author                   134198 non-null  object 
 2   statement                134198 non-null  object 
 3   target                   134198 non-null  bool   
 4   BinaryNumTarget          134198 non-null  float64
 5   manual_keywords          134198 non-null  object 
 6   tweet                    134198 non-null  object 
 7   5_label_majority_answer  134198 non-null  object 
 8   3_label_majority_answer  134198 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 8.3+ MB


In [7]:
df.isna().sum()

Unnamed: 0                 0
author                     0
statement                  0
target                     0
BinaryNumTarget            0
manual_keywords            0
tweet                      0
5_label_majority_answer    0
3_label_majority_answer    0
dtype: int64

In [9]:
df.duplicated().sum()

0

In [12]:
df.shape

(134198, 9)

In [19]:
# Remove the unwanted columns, like index column, target column becasue we have the BinaryNumTarget and 5label_ans as we will work on 3label_ans
df.drop(['Unnamed: 0', 'target','5_label_majority_answer'],axis=1)

,author,statement,BinaryNumTarget,manual_keywords,tweet,3_label_majority_answer
0,D.L. Davis,End of eviction moratorium means millions of A...,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders - 6 Month Update\n\nInfl...,Agree
1,D.L. Davis,End of eviction moratorium means millions of A...,1.0,"Americans, eviction moratorium",@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,Agree
2,D.L. Davis,End of eviction moratorium means millions of A...,1.0,"Americans, eviction moratorium",THE SUPREME COURT is siding with super rich pr...,Agree
3,D.L. Davis,End of eviction moratorium means millions of A...,1.0,"Americans, eviction moratorium",@POTUS Biden Blunders\n\nBroken campaign promi...,Agree
4,D.L. Davis,End of eviction moratorium means millions of A...,1.0,"Americans, eviction moratorium",@OhComfy I agree. The confluence of events rig...,Agree
...,...,...,...,...,...,...
134193,Tom Kertscher,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"Biden, great grandfather, slave owner",Joe Biden's family owned African slaves....\n\...,Agree
134194,Tom Kertscher,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"Biden, great grandfather, slave owner","Joe Bidens great, great grandfather was a slav...",Agree
134195,Tom Kertscher,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"Biden, great grandfather, slave owner","@ChevyChaseToGo ""Joe Bidens great-grandfather ...",Agree
134196,Tom Kertscher,Joe Bidens great-grandfather Joseph J. Biden w...,0.0,"Biden, great grandfather, slave owner",@JoeBiden Facts are Bidens VP Kamala Harris Gr...,Agree
